In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
class binary_TREE():
    def __init__(self):
        self.tree = input('Enter the name of tree you want to use(1 : CART, 2: C4.5) : ')
       
    def fit(self):
        self.__input_train()
        X = self.X
        y = self.y
        ytype = np.sort(y.unique())
        self.ytype = ytype
        self.__CART_fit(X,y)
        self.__C45_fit(X,y)

    def predict(self):
        y_tst, X_tst = self.y , self.X
        ytype = self.ytype

        if self.tree == '1':
            y_pred = self.y_pred_cart
            bestvariable = self.bestvariable_cart
            bestcrit = self.bestcrit_cart

            x_cl_tst = X_tst.iloc[:,bestvariable]

            y_hat = np.zeros(len(y_tst)).astype(type(ytype[0]))


            if self.is_numeric_cart:
                y_hat[x_cl_tst >= bestcrit] = y_pred[0]
                y_hat[x_cl_tst < bestcrit] = y_pred[1]

            else:
                idx = self.__find_subset_index2(bestcrit,x_cl_tst)
                notidx = [x for x in range(len(y_tst)) if not(x in idx)]
                y_hat[idx] = y_pred[0] ; y_hat[notidx] = y_pred[1]

        else:
            y_pred = self.y_pred_c45
            bestvariable = self.bestvariable_c45
            bestcrit = self.bestcrit_c45

            x_cl_tst = X_tst.iloc[:,bestvariable]

            y_hat = np.zeros(len(y_tst)).astype(type(ytype[0]))

            if self.is_numeric_c45:
                y_hat[x_cl_tst >= bestcrit] = y_pred[0]
                y_hat[x_cl_tst < bestcrit] = y_pred[1]

            else:
                index = []
                for i in bestcrit:
                    index.append(x_cl_tst == i)

                for idx in range(len(index)):
                    y_hat[index[idx]] = y_pred[idx]

        result = pd.DataFrame()
        result['Actual Class'] = y_tst
        result['Predicted Class'] = y_hat

        return result
        

    def confusion_matrix_accuracy(self):
        ytype = self.ytype
        result = self.predict()
        C = []
        for i in range(len(ytype)):
            for j in range(len(ytype)):
                C.append(((result['Actual Class'] == ytype[i]) & (result['Predicted Class'] == ytype[j])).sum())

        C2 = pd.DataFrame(np.array(C).reshape(len(ytype),len(ytype)),
                          index = ytype, columns = ytype)
        acc_tst = round(np.diag(C2).sum()/C2.sum().sum(),3)

        return C2, acc_tst

    
    def print_results(self):
        out_name = input("Enter the output file name to export [(ex) result.txt] : ")
        C2, acc_tst = self.confusion_matrix_accuracy()
        ytype = self.ytype
        X = self.X
        
        C2.to_csv('temp2.txt', sep = '\t')
        temp2 = []
        temp2.append('\tPredicted Class \n')
        r = open('temp2.txt','r')
        for line in r.readlines():
            temp2.append(line)
        r.close()    
        for i in range(len(temp2)):
            if i == 2:
                temp2[i] = 'Actual \t' + temp2[i]+ '\n'
            elif i == 3:
                temp2[i] = 'Class \t' + temp2[i]+ '\n'
            else:
                temp2[i] = '\t' + temp2[i]+ '\n'


        f = open(out_name, 'w+')
        f.write('Tree Structure (CART)  \n')
        if self.is_numeric_cart:
            f.write(f'\tNode 1 : x{self.bestvariable_cart+1} >= {self.bestcrit_cart} ({(self.y == ytype[0]).sum()},{(self.y == ytype[1]).sum()}) \n')
            f.write(f'\t\tNode 2 : {self.y_pred_cart[0]} ({(self.y_classified_cart[0] == ytype[0]).sum()},{(self.y_classified_cart[0] == ytype[1]).sum()}) \n')
            f.write(f'\t\tNode 3 : {self.y_pred_cart[1]} ({(self.y_classified_cart[1] == ytype[0]).sum()},{(self.y_classified_cart[1] == ytype[1]).sum()}) \n')
        else:
            if ((self.y == ytype[0]).sum())  / len(self.y) >= 0.5:
                f.write(f'\tNode 1 : {ytype[0]} ({(self.y == ytype[0]).sum()}, {(self.y == ytype[1]).sum()} ) \n')
            else:
                f.write(f'\tNode 1 : {ytype[1]} ({(self.y == ytype[0]).sum()}, {(self.y == ytype[1]).sum()} ) \n')
            
            f.write(f'\t\tNode 2 : Class = {self.bestcrit_cart}, {self.y_pred_cart[0]} ({(self.y_classified_cart[0] == ytype[0]).sum()},{(self.y_classified_cart[0] == ytype[1]).sum()}) \n')
            f.write(f'\t\tNode 3 : Class = {[x for x in np.sort(X.iloc[:,self.bestvariable_cart].unique()) if not(x in self.bestcrit_cart)]}, {self.y_pred_cart[1]} ({(self.y_classified_cart[1] == ytype[0]).sum()},{(self.y_classified_cart[1] == ytype[1]).sum()}) \n')

            
        f.write('Tree Structure (C4.5)  \n')
        if self.is_numeric_c45:
            f.write(f'\tNode 1 : x{self.bestvariable_c45+1} >= {self.bestcrit_c45} ({(self.y == ytype[0]).sum()},{(self.y == ytype[1]).sum()}) \n')
            f.write(f'\t\tNode 2 : {self.y_pred_c45[0]} ({(self.y_classified_c45[0] == ytype[0]).sum()},{(self.y_classified_c45[0] == ytype[1]).sum()}) \n')
            f.write(f'\t\tNode 3 : {self.y_pred_c45[1]} ({(self.y_classified_c45[1] == ytype[0]).sum()},{(self.y_classified_c45[1] == ytype[1]).sum()}) \n')
        else:
            if ((self.y == ytype[0]).sum())  / len(self.y) >= 0.5:
                f.write(f'\tNode 1 : {ytype[0]} ({(self.y == ytype[0]).sum()}, {(self.y == ytype[1]).sum()} ) \n')
            else:
                f.write(f'\tNode 1 : {ytype[1]} ({(self.y == ytype[0]).sum()}, {(self.y == ytype[1]).sum()} ) \n')
            
            for i in range(len(self.bestcrit_c45)):
                f.write(f'\t\tNode {i+2} : Class = {[self.bestcrit_c45[i]]} {self.y_pred_c45[i]} ({(self.y_classified_c45[i] == ytype[0]).sum()}, {(self.y_classified_c45[i] == ytype[1]).sum()}) \n')                                
            
        f.write('\n\n')
        f.write('Confusion Matrix (Train)\n')
        f.write('---------------------------------\n')
        for line in temp2:
            f.write(line)
        f.write('\n')
        f.write('Model Summary (Train) \n')
        f.write('---------------------------------\n')
        f.write(f'Overall accuracy = {acc_tst}')
        f.close()
        
        r = open(out_name,'r')
        for line in r.readlines():
            print(line)
        r.close()
                
        os.remove('temp2.txt')        
        
        pass
    

    def __input_train(self):
        # input data file name
        train_data=input("Enter the name of training data file [(ex) titanic.csv] : ") # data name
        self.train_data = train_data
        coding_fm=int(input("Select the data Seperator(1 = ' ' or 2 = ','): ")) # data separator of training data
        self.coding_fm = coding_fm
        separator_fm={coding_fm ==1 : " "}.get(True, ",")
        self.separator_fm = separator_fm

        # input header & assign response variables
        self.header=input("Does the data have column header? (y/n) : ")
        header = self.header
        if(self.header=="y") :
            trdata=pd.read_csv(train_data, sep=separator_fm) # loading data with header
            self.trdata = trdata
            res_pos = input(f'Enter the column name of response variable among {list(trdata.columns)} : ')
            self.res_pos = res_pos           
            self.y = trdata[res_pos]
            self.X = trdata.drop(self.res_pos, axis = 1)

        else : 
            self.trdata=pd.read_csv(train_data, sep=separator_fm, header=None) # loading data without header
            trdata = self.trdata
            self.res_pos = int(input(f'Enter the column position of the response variable : \n [from 1 to {trdata.shape[1]}] : '))
            res_pos = self.res_pos
            X_index = []
            for i in range(len(trdata.columns)): 
                if i == res_pos-1 : continue
                X_index.append(i)
            self.y = trdata.iloc[:,res_pos-1]
            self.X = trdata.iloc[:,X_index]
    
    
    def __input_test(self):
        test_data =input("Enter the name of test data file [(ex) titanic.csv] : ")
        coding_fm = self.coding_fm
        separator_fm = self.separator_fm
        header = self.header
        res_pos = self.res_pos
        
        if(self.header=="y") :
            tstdata=pd.read_csv(test_data, sep=separator_fm) # loading data with header
            y_tst = tstdata[res_pos]
            X_tst = tstdata.drop(self.res_pos, axis = 1)

        else : 
            tstdata=pd.read_csv(test_data, sep=separator_fm, header=None) # loading data without header
            X_index = []
            for i in range(len(tstdata.columns)): 
                if i == res_pos-1 : continue
                X_index.append(i)
            y_tst = tstdata.iloc[:,res_pos-1]
            X_tst = tstdata.iloc[:,X_index]
        
        return y_tst, X_tst

    def __Gini_impurity(self,X,y,crt):
        crt_bigger = X >= crt
        crt_smaller = X < crt
        ytype = self.ytype

        g1_p_sq = np.array([((y[crt_bigger] == ytype[i]).sum() / len(y[crt_bigger])) ** 2 for i in range(len(ytype))])
        g1_impurity = 1 - g1_p_sq.sum()

        g2_p_sq = np.array([((y[crt_smaller] == ytype[i]).sum() / len(y[crt_smaller])) ** 2 for i in range(len(ytype))])
        g2_impurity = 1 - g2_p_sq.sum()

        w1 = len(y[crt_bigger]) / len(y)
        w2 = len(y[crt_smaller]) / len(y)

        impurity = w1 * g1_impurity + w2 * g2_impurity
        return impurity
    
    def __Gini_impurity_nominal(self,y,idx):
        ytype = self.ytype
        notidx = [x for x in range(len(y)) if not(x in idx)]

        g1_p_sq = np.array([((y[idx] == ytype[i]).sum() / len(y[idx])) ** 2 for i in range(len(ytype))])
        g1_impurity = 1 - g1_p_sq.sum()

        g2_p_sq = np.array([((y[notidx] == ytype[i]).sum() / len(y[notidx])) ** 2 for i in range(len(ytype))])
        g2_impurity = 1 - g2_p_sq.sum()

        w1 = len(y[idx]) / len(y)
        w2 = len(y[notidx]) / len(y)

        impurity = w1 * g1_impurity + w2 * g2_impurity
        return impurity

    def __all_subset(self,xtype):
        listsub = list(xtype)
        subsets = []
        for i in range(2**len(listsub)):
            subset = []
            for k in range(len(listsub)):            
                if i & 1<<k:
                    subset.append(listsub[k])
            subsets.append(subset)        
        return subsets[1:len(subsets)-1]

    def __find_subset_index(self,tsubset, t):
        subset_index = []
        for i in range(len(tsubset)):
            temp = []
            for j in tsubset[i]:
                [temp.append(k) for k in list(t[t==j].index)]
                temp.sort()
            subset_index.append(temp)
        return subset_index
    
    def __find_subset_index2(self,tsubset, t):
        subset_index = []
        for j in tsubset:
            [subset_index.append(k) for k in list(t[t==j].index)]
            subset_index.sort()
        return subset_index

    def __best_impurity_for_numeric(self,x_cl,y):
        x_cl_sort = np.sort(x_cl.unique())

        # make criteria of this X variable
        classification_criteria = []
        for i in range(len(x_cl_sort)-1):
            classification_criteria.append(np.median([x_cl_sort[i],x_cl_sort[i+1]]))

        # find best impurity criteria of this X variable
        impurity_hist = []
        for crt in classification_criteria:
            impurity_hist.append(self.__Gini_impurity(x_cl,y,crt))

        crit = classification_criteria[np.argmin(impurity_hist)]
        impurity = impurity_hist[np.argmin(impurity_hist)]

        return crit, impurity

    def __best_impurity_for_nominal(self,x_cl,y):
        xtype = list(np.sort(x_cl.unique()))
        x_cl_subsets = self.__all_subset(xtype)
        x_cl_subsets_index = self.__find_subset_index(x_cl_subsets,x_cl)

        impurity_hist = []
        for i in range(len(x_cl_subsets)):
            impurity_hist.append(self.__Gini_impurity_nominal(y,x_cl_subsets_index[i]))
        crit = x_cl_subsets[np.argmin(impurity_hist)]
        impurity = impurity_hist[np.argmin(impurity_hist)]

        return crit,impurity
    
    def __CART_fit(self,X,y):
        ytype = self.ytype
        crit = []
        impurity = []
        catg = list(X.dtypes)
        for i in range(len(X.columns)):
            x_cl = X.iloc[:,i]
            if not(catg[i] == 'O'):
                temp = self.__best_impurity_for_numeric(x_cl,y)
                crit.append(temp[0]) ; impurity.append(temp[1])
            else:
                temp = self.__best_impurity_for_nominal(x_cl,y)
                crit.append(temp[0]) ; impurity.append(temp[1])

        #### self로 저장할 것들 ####
        bestvariable= np.argmin(impurity)
        bestcrit = crit[bestvariable]
        bestimpurity = impurity[bestvariable]
        bestX = X.iloc[:,bestvariable]
        is_numeric = not(catg[bestvariable] == 'O')


        y_classified = []
        if is_numeric:
            y_classified.append(y[bestX >= bestcrit])
            y_classified.append(y[bestX < bestcrit])
        else:
            idx = self.__find_subset_index2(bestcrit,bestX)
            notidx = [x for x in range(len(y)) if not(x in idx)]
            y_classified.append(y[idx])
            y_classified.append(y[notidx])

        prob = []
        prob.append((y_classified[0] == ytype[0]).sum() / len(y_classified[0]))
        prob.append((y_classified[1] == ytype[0]).sum() / len(y_classified[1]))

        y_pred = []
        for i in range(len(prob)):
            if prob[i] >= 0.5 :
                y_pred.append(ytype[0])
            else :
                y_pred.append(ytype[1])

        self.bestvariable_cart = bestvariable
        self.bestcrit_cart = bestcrit
        self.bestimpurity_cart = bestimpurity
        self.is_numeric_cart = is_numeric
        self.y_classified_cart = y_classified
        self.prob_cart = prob
        self.y_pred_cart = y_pred
        pass

    def __Entrophy(self,X,y,crt):
        crt_bigger = X >= crt
        crt_smaller = X < crt
        ytype = self.ytype

        g1_p = np.array([(y[crt_bigger] == ytype[i]).sum() / len(y[crt_bigger]) for i in range(len(ytype))])
        log_g1_p = []
        for i in g1_p:
            if i == 0:
                log_g1_p.append(0)
            else:
                log_g1_p.append(np.log2(i))
        g1_ent = -np.sum(g1_p * log_g1_p)

        g2_p = np.array([(y[crt_smaller] == ytype[i]).sum() / len(y[crt_smaller]) for i in range(len(ytype))])
        log_g2_p = []
        for i in g2_p:
            if i == 0:
                log_g2_p.append(0)
            else:
                log_g2_p.append(np.log2(i))
        g2_ent = -np.sum(g2_p * log_g2_p)

        w1 = len(y[crt_bigger]) / len(y)
        w2 = len(y[crt_smaller]) / len(y)

        entrophy = w1 * g1_ent + w2 * g2_ent

        intrinsic = -(w1 * np.log2(w1) + w2*np.log2(w2))


        return entrophy, intrinsic
    
    def __Entrophy_nominal(self,x_cl,y):
        ytype = self.ytype
        xtype = np.sort(x_cl.unique())
        index = []
        for i in xtype:
            index.append(x_cl == i)

        g_p = []
        for i in index:
            for j in range(len(ytype)):
                g_p.append((y[i] == ytype[j]).sum() / len(y[i]))

        g_p = np.array(g_p).reshape((len(xtype),len(ytype)))
        log_g_p = np.zeros((len(xtype),len(ytype)))
        for i in range(len(xtype)):
            for j in range(len(self.ytype)):
                if g_p[i,j] == 0:
                    log_g_p[i,j] = 0
                else:
                    log_g_p[i,j] = np.log2(g_p[i,j])
        g_ent = -np.sum(g_p*log_g_p,axis = 1)
        w = []
        for i in range(len(xtype)):
            w.append(len(y[index[i]]) / len(y))


        entrophy = np.sum(w * g_ent)

        intrinsic=[]
        for i in range(len(w)):
            intrinsic.append(w[i] * np.log2(w[i]))
        intrinsic = -np.sum(intrinsic)


        return entrophy, intrinsic

    def __best_Entrophy_for_numeric(self,x_cl,y):
        x_cl_sort = np.sort(x_cl.unique())

        # make criteria of this X variable
        classification_criteria = []
        for i in range(len(x_cl_sort)-1):
            classification_criteria.append(np.median([x_cl_sort[i],x_cl_sort[i+1]]))

        # find best Entrophy criteria of this X variable
        entrophy_hist = []
        intrinsic_hist = []
        for crt in classification_criteria:
            temp = self.__Entrophy(x_cl,y,crt)
            entrophy_hist.append(temp[0]) ; intrinsic_hist.append(temp[1])
        best_index = np.argmin(np.array(entrophy_hist) / np.array(intrinsic_hist))
        crit = classification_criteria[best_index]
        entrophy = entrophy_hist[best_index]
        intrinsic = intrinsic_hist[best_index]

        return crit, entrophy, intrinsic
    
    def __C45_fit(self,X,y):
        ytype = self.ytype
        crit = []
        entrophy = []
        intrinsic_info = []
        catg = list(X.dtypes)
        for i in range(len(X.columns)):
            x_cl = X.iloc[:,i]
            if not(catg[i] == 'O'):
                temp = self.__best_Entrophy_for_numeric(x_cl,y)
                crit.append(temp[0]) ; entrophy.append(temp[1]) ; intrinsic_info.append(temp[2])
            else:
                crit.append(list(np.sort(x_cl.unique())))
                temp = self.__Entrophy_nominal(x_cl,y)
                entrophy.append(temp[0])
                intrinsic_info.append(temp[1])

        #Gain
        gain_p = []
        for i in range(len(ytype)):
            gain_p.append((y == ytype[i]).sum() / len(y))

        gain_ent = -np.sum(gain_p * np.log2(gain_p))

        gain_ratio = (gain_ent - entrophy) / intrinsic_info


        bestvariable= np.argmax(gain_ratio)
        bestcrit = crit[bestvariable]
        bestgainratio = gain_ratio[bestvariable]
        bestX = X.iloc[:,bestvariable]
        is_numeric = not(catg[bestvariable] == 'O')


        y_classified = []
        if is_numeric:
            y_classified.append(y[bestX >= bestcrit])
            y_classified.append(y[bestX < bestcrit])
        else:
            index = []
            for i in bestcrit:
                index.append(bestX == i)
            for idx in index:
                y_classified.append(y[idx])

        prob = []
        prob.append((y_classified[0] == ytype[0]).sum() / len(y_classified[0]))
        prob.append((y_classified[1] == ytype[0]).sum() / len(y_classified[1]))

        y_pred = []
        for i in range(len(prob)):
            if prob[i] >= 0.5 :
                y_pred.append(ytype[0])
            else :
                y_pred.append(ytype[1])


        self.bestvariable_c45 = bestvariable
        self.bestcrit_c45 = bestcrit
        self.bestgainratio_c45 = bestgainratio
        self.is_numeric_c45 = is_numeric
        self.y_classified_c45 = y_classified
        self.prob_c45 = prob
        self.y_pred_c45 = y_pred    
        pass

### Cart for numeric, nomial predictor variables,
### C4.5 for numeric, nomial predictor variables works

In [4]:
c45_nominal = binary_TREE()

Enter the name of tree you want to use(1 : CART, 2: C4.5) :  2


In [5]:
c45_nominal.fit()

Enter the name of training data file [(ex) titanic.csv] :  titanic.csv
Select the data Seperator(1 = ' ' or 2 = ','):  2
Does the data have column header? (y/n) :  y
Enter the column name of response variable among ['Class', 'Age', 'Sex', 'Survived'] :  Survived


In [6]:
c45_nominal.predict()

,Actual Class,Predicted Class
0,Yes,No
1,Yes,No
2,Yes,No
3,Yes,No
4,Yes,No
...,...,...
2196,Yes,Yes
2197,Yes,Yes
2198,No,Yes
2199,No,Yes


In [7]:
c45_nominal.confusion_matrix_accuracy()

(       No  Yes
 No   1364  126
 Yes   367  344,
 0.776)

In [8]:
c45_nominal.print_results()

Enter the output file name to export [(ex) result.txt] :  result.txt


Tree Structure (CART)  

	Node 1 : No (1490, 711 ) 

		Node 2 : Class = ['Female'], Yes (126,344) 

		Node 3 : Class = ['Male'], No (1364,367) 

Tree Structure (C4.5)  

	Node 1 : No (1490, 711 ) 

		Node 2 : Class = ['Female'] Yes (126, 344) 

		Node 3 : Class = ['Male'] No (1364, 367) 





Confusion Matrix (Train)

---------------------------------

		Predicted Class 



		No	Yes



Actual 	No	1364	126



Class 	Yes	367	344





Model Summary (Train) 

---------------------------------

Overall accuracy = 0.776
